# Obtenção dos Dados

## Origem dos dados

In [ ]:
# !pip install kagglehub -U

# !pip install pandas -U

# !pip install fastparquet

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print(path)


In [ ]:
import shutil
import os

arquivos = os.listdir(path)
print(arquivos)

for arquivo in arquivos:
    source = os.path.join(path, arquivo)
    destination = os.path.join(os.getcwd(), arquivo)
    shutil.move(source, destination)

## Extração dos Dados

### Importação dos CSV's dentro de um dictionary de dataframes

In [ ]:
import pandas as pd
import os
from pathlib import Path

pd.set_option('display.max_columns', None)

def create_dataframes():
    # Dictionary to store our dataframes
    df_dict = {}

    print(os.getcwd())

    csv_files = os.listdir(os.getcwd())
    
    for file in csv_files:
        if file.endswith("csv"):
            # Create variable name from filename
            # Remove file extension and any special characters
            df_name = Path(file.replace('olist','').replace('dataset','')).stem  # Gets filename without extension
            df_name = ''.join(c for c in df_name if c.isalnum())  # Remove special chars
            
            globals()[df_name] = pd.read_csv(file)
            
            # Also store in dictionary
            df_dict[df_name] = globals()[df_name]
            
            print(f"Created DataFrame: {df_name}")
            print(f"Shape: {globals()[df_name].shape}")
            print("-" * 50)

    return df_dict

dataframes = create_dataframes()




In [ ]:
csv_files = os.listdir(os.getcwd())

for file in csv_files:
    if file.endswith("parquet"):
        os.remove(file)

In [ ]:
for name in dataframes.keys():
    print(f'Dataset: {name}')
    dataframes[name].to_parquet(f"{name}.parquet", index=False, engine='fastparquet')

In [ ]:
csv_files = os.listdir(os.getcwd())

for file in csv_files:
    if file.endswith("csv"):
        os.remove(file)

#### Primeira visualição dos dados

In [ ]:
for name in dataframes.keys():
    print(f'Dataset: {name}')
    display(dataframes[name].head())
    display(dataframes[name].info())    

In [ ]:
for name in dataframes.keys():
    print(f'Dataset: {name}')
    display(dataframes[name].columns)


### Primeiros tratamentos e merge de dataframes

In [ ]:
dataframes['geolocation']

In [ ]:
df_geo_seller = dataframes['geolocation'].copy()

df_geo_seller.columns = ['seller_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'seller_city', 'seller_state']

df_seller = pd.merge(dataframes['sellers'], df_geo_seller,how='left',on=['seller_zip_code_prefix','seller_city','seller_state'])

df_seller.head()

In [ ]:
df_seller.to_parquet('sellers.parquet',compression='snappy',engine='fastparquet')

In [ ]:
df_geo_customer = dataframes['geolocation'].copy()

df_geo_customer.columns = ['customer_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'customer_city', 'customer_state']

df_customer = pd.merge(dataframes['customers'], df_geo_customer,how='left',on=['customer_zip_code_prefix','customer_city','customer_state'])
df_customer.to_parquet('customers.parquet',compression='snappy',engine='fastparquet')

df_customer.head()

In [ ]:
df_orders = pd.merge(dataframes['orders'],dataframes['orderpayments'],how='left',on='order_id')

df_orders = pd.merge(df_orders,dataframes['orderreviews'],how='left',on='order_id')

df_orders = pd.merge(df_orders,df_customer,how='left',on='customer_id')



In [ ]:
df_orderitems = pd.read_parquet('orderitems.parquet',engine='fastparquet')
df_products = pd.read_parquet('products.parquet',engine='fastparquet')
df_seller = pd.read_parquet('sellers.parquet',engine='fastparquet')

df_orderitems = pd.merge(df_orderitems, df_products, how='left', on='product_id')
df_orderitems = pd.merge(df_orderitems, df_seller, how='left', on='seller_id')

df_orderitems.head()

In [ ]:
df_orderitems['shipping_limit_date'] = pd.to_datetime(df_orderitems['shipping_limit_date'])
df_orderitems.to_parquet('orders_item.parquet',compression='snappy',engine='fastparquet')

In [ ]:
df_orders.head()

In [ ]:
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'])
df_orders['order_estimated_delivery_date'] = pd.to_datetime(df_orders['order_estimated_delivery_date'])
df_orders['review_creation_date'] = pd.to_datetime(df_orders['review_creation_date'])
df_orders['review_answer_timestamp'] = pd.to_datetime(df_orders['review_answer_timestamp'])
df_orders.to_parquet('orders.parquet',compression='snappy',engine='fastparquet')

In [ ]:
df_orders.info()

In [ ]:
df_orders.isna().sum()